# Single view

In [2]:
from tqdm import tqdm
from util import *
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import os
import pandas as pd
import copy
from PIL import Image
%matplotlib inline

In [3]:
tr = pd.read_csv('./combine_txt/sun397_train_lt.txt', header=None, sep=' ')

In [4]:
class_names = {}
class_num = {}

In [5]:
for lab in tr[1].unique():
    temp = tr.loc[tr[1] == lab].iloc[0, 0]
    class_name = temp.split('/')[-2]
    class_names[lab] = class_name
    class_num[lab] = len(tr.loc[tr[1] == lab])

In [6]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [7]:
#  device = torch.device('cpu')

In [8]:
LOG_DIR = './log'
DATASET = 'plain'
MODEL_ID = 'test'
DATALOADER_WORKERS = 4
LEARNING_RATE = 0.01
LR_DECAY_FACTOR = 0.1
LR_DECAY_EPOCHS = 10
DROPOUT = False
DROPOUT_RATE = 0.5
MOMENTUM = 0.9
EPOCHS = 1
BATCH_SIZE = 256
DISPLAY_STEP = 10
NUM_CLASSES = 397

if not os.path.isdir(LOG_DIR):
    os.makedirs(LOG_DIR)

In [9]:
class sun_dataset_test (torch.utils.data.Dataset):
    
    def __init__ (self, txt_file, transform=None):
        super().__init__()
        self.df = pd.read_csv(txt_file, header=None, sep=' ')
        self.transform = transform
        
    def __len__ (self):
        return len(self.df)
    
    def __getitem__ (self, idx):
        
        image = Image.open(self.df.iloc[idx, 0])
        label = self.df.iloc[idx, 1] - 1
#         image_dir = self.df.iloc[idx, 0]
        
        if self.transform:
            image = self.transform(image)
            
        return image, label

In [10]:
# transforms.RandomResizeCrop(224)
data_transform = transforms.Compose([
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

In [11]:
dataset = sun_dataset_test(txt_file='./sun397_test_lt.txt', transform=data_transform)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=DATALOADER_WORKERS)
dataset_size = len(dataset)

In [15]:
checkpoint = torch.load('./log/model_combine_aug_base_lr_100_ep_150_caffe_drop_0.5_dim_4096_checkpoint.pth.tar')
DROPOUT = True
DROPOUT_RATE = 0.5
FC_ADD_DIM = 4096
BIAS = True

In [16]:
resnet = torchvision.models.resnet152(pretrained=True)
# Freeze all layers
for param in resnet.parameters():
    param.requires_grad = False

print('Done.')    

if DROPOUT:
    print('Model using dropout.')
    resnet = MyResNet(resnet, NUM_CLASSES, DROPOUT_RATE, FC_ADD_DIM, bias=BIAS)
else:
    print('Model not using dropout.')
    # Reset the fc layer
    num_features = resnet.fc.in_features
    resnet.fc = nn.Linear(num_features, NUM_CLASSES)

resnet = resnet.to(device)

loss_function = nn.CrossEntropyLoss()

Done.
Model using dropout.
Bias is True
Intermediate fc dimension is: 4096
Dropout rate is: 0.500000


In [17]:
model = resnet

model.eval()

model.load_state_dict(checkpoint['state_dict'])

In [18]:
running_loss = 0.0
running_correct_total = 0

class_correct = torch.tensor([0. for i in range(NUM_CLASSES)])
class_total = torch.tensor([0. for i in range(NUM_CLASSES)])

# Iterate over data
for inputs, labels in tqdm(dataloader):

    inputs, labels = inputs.to(device), labels.to(device)

    # Forward
    # If on training phase, enable gradients
    with torch.set_grad_enabled(False):

        logits = model(inputs)
        _, preds = torch.max(logits, 1)
        loss = loss_function(logits, labels)
    
    # Record loss and correct predictions
    correct_tensor = (preds == labels).squeeze()
    running_loss += loss.item() * inputs.shape[0]
    running_correct_total += correct_tensor.sum().item()

    for i in range(len(labels)):
        label = labels[i]
        class_correct[label] += correct_tensor[i].item()
        class_total[label] += 1

# Epoch loss and accuracieds
test_loss = running_loss / dataset_size
test_acc_mic = running_correct_total / dataset_size
test_acc_mac = (class_correct / class_total).mean().item()

print('loss: %.3f, accuracy_micro: %.3f, accuracy_macro: %.3f' 
      % (test_loss, test_acc_mic, test_acc_mac))
    

100%|██████████| 63/63 [01:06<00:00,  1.05s/it]

loss: 2.925, accuracy_micro: 0.480, accuracy_macro: 0.480


In [58]:
# for l in range(len(class_acc)):
#     print('class id: %4d,    class name: %25s,    class data number: %5d,    accuracy: %.1f' %(l, class_names[l+1], class_num[l+1], class_acc[l]*100))

plain_base_lr_40_ep_70_caffe_drop_0.5_dim_4096 0.451

plain_base_lr_30_ep_70_caffe_drop_0.5_dim_4096 0.448

plain_base_lr_30_ep_70_caffe_drop_0.5_dim_4096_zerobias 0.447

plain_base_lr_30_ep_70_caffe_drop_0.5_dim_2048 0.444

plain_base_lr_30_ep_70_caffe_drop_0.5_dim_1024 0.437



plain_base_lr_30_ep_70_caffe_drop_0.3_dim_4096 0.443

plain_base_lr_30_ep_70_caffe_drop_0.3_dim_2048 0.445

plain_base_lr_30_ep_70_caffe_drop_0.3_dim_1024 0.444


plain_base_lr_50_ep_150_caffe_drop_0.5_dim_4096 0.448

plain_base_lr_60_ep_150_caffe_drop_0.5_dim_4096 0.452

plain_base_lr_70_ep_150_caffe_drop_0.5_dim_4096 0.452

plain_base_lr_100_ep_150_caffe_drop_0.5_dim_4096 0.453 113



combine_base_lr_70_ep_150_caffe_drop_0.5_dim_4096 0.472 136

aug_base_lr_100_ep_150_caffe_drop_0.5_dim_4096 0.457 99

combine_aug_sm_base_lr_100_ep_150_caffe_drop_0.5_dim_4096 0.477 126

combine_aug_base_lr_100_ep_150_caffe_drop_0.5_dim_4096 0.48 110

python ResNet_SUN397.py --dataset plain --epoch 150 --model_id plain_base_lr_70_ep_150_caffe_drop_0.5_dim_4096 --decay_epoch 70 --caffe_weights True --dropout True --dropout_rate 0.5 --fc_add_dim 4096